In [188]:
#Local Search and Simulated Annealing

In [228]:
import sys
import copy
import math
import random

sys.path.append(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence')
from sudoku_parser import SudokuPuzzle

puzzle = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Hard-P1.txt')
puzzle.display()
# Step 1: Create fixed_cells matrix
fixed_cells = [[False]*9 for _ in range(9)]
for r in range(9):
    for c in range(9):
        if puzzle.grid[r][c] != 0:
            fixed_cells[r][c] = True


? ? 4 ? ? 8 ? ? 6
? ? ? ? ? 7 8 3 ?
? 7 ? 3 ? ? ? ? 9
? ? 7 ? ? ? 5 ? 3
2 4 ? ? ? ? ? 9 7
9 ? 6 ? ? ? 2 ? ?
6 ? ? ? ? 5 ? 7 ?
? 8 9 6 ? ? ? ? ?
4 ? ? 8 ? ? 3 ? ?


In [229]:
def populate_random_by_block(puzzle):
    for block_row in range(3):
        for block_col in range(3):
            # Collect fixed numbers in this block
            fixed = set()
            empty_positions = []
            for r in range(block_row * 3, block_row * 3 + 3):
                for c in range(block_col * 3, block_col * 3 + 3):
                    val = puzzle.grid[r][c]
                    if val != 0:
                        fixed.add(val)
                    else:
                        empty_positions.append((r, c))
            
            # Numbers missing in this block
            missing = list(set(range(1, 10)) - fixed)
            random.shuffle(missing)
            
            # Fill empty cells with missing numbers randomly
            for pos, num in zip(empty_positions, missing):
                puzzle.set_digit(pos[0], pos[1], num)


In [230]:
def swap_two_cells_in_block(puzzle, fixed_cells):
    block_row = random.randint(0, 2)
    block_col = random.randint(0, 2)

    candidates = []
    for r in range(block_row * 3, block_row * 3 + 3):
        for c in range(block_col * 3, block_col * 3 + 3):
            if not fixed_cells[r][c]:
                candidates.append((r, c))

    if len(candidates) < 2:
        return  # nothing to swap

    (r1, c1), (r2, c2) = random.sample(candidates, 2)
    puzzle.grid[r1][c1], puzzle.grid[r2][c2] = puzzle.grid[r2][c2], puzzle.grid[r1][c1]


In [231]:
def local_search_solver(puzzle, fixed_cells, max_steps=50000):
    puzzle.populate_random()  # Fill in random numbers
    current_conflicts = puzzle.amount_of_comflicts()
    
    for step in range(max_steps):
        if current_conflicts == 0:
            print(f"Solved in {step} steps!")
            break
        
        new_puzzle = copy.deepcopy(puzzle)
        swap_two_cells_in_block(new_puzzle, fixed_cells)  # pass fixed_cells here
        
        new_conflicts = new_puzzle.amount_of_comflicts()
        
        if new_conflicts < current_conflicts:
            puzzle = new_puzzle
            current_conflicts = new_conflicts
    
    puzzle.display()
    print("Final Conflicts:", current_conflicts)


In [232]:
def simulated_annealing(puzzle, fixed_cells, max_steps=50000, initial_temp=10.0, cooling_rate=0.995):
    populate_random_by_block(puzzle)
    current_conflicts = puzzle.amount_of_comflicts()
    temp = initial_temp
    
    for step in range(max_steps):
        if current_conflicts == 0:
            print(f"Solved in {step} steps!")
            break
        
        new_puzzle = copy.deepcopy(puzzle)
        swap_two_cells_in_block(new_puzzle, fixed_cells)  # mutate
        new_conflicts = new_puzzle.amount_of_comflicts()

        delta = new_conflicts - current_conflicts
        if delta < 0 or random.uniform(0, 1) < math.exp(-delta / temp):
            puzzle = new_puzzle
            current_conflicts = new_conflicts

        temp *= cooling_rate

    puzzle.display()
    print("Final Conflicts:", current_conflicts)


In [233]:
# Run Local Search
import copy
print("Running Local Search...")
local_search_solver(copy.deepcopy(puzzle), fixed_cells)

# Run Simulated Annealing
print("\nRunning Simulated Annealing...")
simulated_annealing(copy.deepcopy(puzzle), fixed_cells)


Running Local Search...
1 6 4 2 4 8 9 3 6
1 4 9 2 3 7 8 3 5
1 7 6 3 5 8 4 4 9
9 9 7 1 2 2 5 2 3
2 4 1 5 1 6 9 9 7
9 3 6 8 7 1 2 9 5
6 1 3 8 9 5 6 7 2
7 8 9 6 3 3 1 5 3
4 2 4 8 8 5 3 8 9
Final Conflicts: 60

Running Simulated Annealing...
3 9 4 2 5 8 7 1 6
5 6 1 9 4 7 8 3 2
8 7 2 3 6 1 4 5 9
1 3 7 4 9 6 5 8 3
2 4 8 5 1 2 6 9 7
9 5 6 7 8 3 2 4 1
6 1 3 1 2 5 9 7 8
7 8 9 6 3 4 1 2 5
4 2 5 8 7 9 3 6 4
Final Conflicts: 4


In [234]:
def select_variable_by_degree(puzzle):
    unassigned = puzzle.available_indexes()
    if not unassigned:
        return None  # This is where your None is coming from

    max_degree = -1
    selected_var = None

    for row, col in unassigned:
        neighbors = get_neighbors(row, col)
        degree = sum(1 for r, c in neighbors if puzzle.grid[r][c] == 0)

        if degree > max_degree:
            max_degree = degree
            selected_var = (row, col)

    return selected_var


In [235]:
def select_min_conflict_value(puzzle: SudokuPuzzle, row, col):
    min_conflicts = float('inf')
    best_value = None
    for value in range(1, 10):
        puzzle.set_digit(row, col, value)
        conflicts = puzzle.amount_of_comflicts()
        if conflicts < min_conflicts:
            min_conflicts = conflicts
            best_value = value
        puzzle.set_digit(row, col, 0)  # Reset
    return best_value


In [236]:
def simulated_annealing_solver(puzzle: SudokuPuzzle, initial_temp=1.0, cooling_rate=0.99, max_steps=10000):
    populate_random_by_block(puzzle)
    puzzle.set_domains()
    current_conflicts = puzzle.amount_of_comflicts()
    temperature = initial_temp

    for step in range(max_steps):
        if current_conflicts == 0:
            break

        selected = select_variable_by_degree(puzzle)
        if selected is None:
            break  # No unassigned variable to select

        row, col = selected
        old_value = puzzle.grid[row][col]
        new_value = select_min_conflict_value(puzzle, row, col)

        if new_value is None:
            continue

        puzzle.set_digit(row, col, new_value)
        new_conflicts = puzzle.amount_of_comflicts()
        delta = new_conflicts - current_conflicts

        if delta < 0 or random.uniform(0, 1) < math.exp(-delta / temperature):
            current_conflicts = new_conflicts  # Accept the new state
        else:
            puzzle.set_digit(row, col, old_value)  # Revert

        temperature *= cooling_rate

    return current_conflicts == 0


In [237]:
def populate_random_from_domains(puzzle: SudokuPuzzle):
    for row in range(9):
        for col in range(9):
            if puzzle.grid[row][col] == 0:
                domain = puzzle.domains[row][col]
                if domain:
                    puzzle.set_digit(row, col, random.choice(domain))


In [238]:
def main():
    import math
    max_attempts = 5
    solved = False

    for attempt in range(1, max_attempts + 1):
        print(f"\n🔁 Attempt {attempt} — Running Simulated Annealing...")

        # Reload puzzle fresh each time
        puzzle = SudokuPuzzle(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Hard-P1.txt')
        puzzle.set_domains()
        populate_random_from_domains(puzzle)  # Initialize from domains

        # Run simulated annealing solver
        success = simulated_annealing_solver(
            puzzle,
            initial_temp=10.0,
            cooling_rate=0.995,
            max_steps=50000
        )

        # Display result
        puzzle.display()
        print("Final Conflicts:", puzzle.amount_of_comflicts())

        if success:
            print("✅ Puzzle Solved!")
            solved = True
            break
        else:
            print("❌ Could not solve the puzzle on this attempt.")

    if not solved:
        print("\n🚫 Simulated Annealing failed after all attempts.")


In [239]:
if __name__ == "__main__":
    main()



🔁 Attempt 1 — Running Simulated Annealing...
1 2 4 2 9 8 7 2 6
1 5 1 2 1 7 8 3 5
1 7 5 3 5 4 1 4 9
1 1 7 4 1 4 5 8 3
2 4 5 1 8 6 1 9 7
9 3 6 7 4 3 2 1 8
6 2 2 1 9 5 4 7 4
3 8 9 6 2 2 1 4 4
4 5 5 8 9 9 3 1 1
Final Conflicts: 59
❌ Could not solve the puzzle on this attempt.

🔁 Attempt 2 — Running Simulated Annealing...
5 3 4 5 9 8 7 5 6
5 9 1 1 6 7 8 3 1
1 7 1 3 1 2 4 1 9
1 1 7 1 8 1 5 1 3
2 4 1 1 6 1 1 9 7
9 1 6 5 1 4 2 4 1
6 1 3 1 2 5 1 7 8
5 8 9 6 1 3 4 2 4
4 5 5 8 9 2 3 2 1
Final Conflicts: 66
❌ Could not solve the puzzle on this attempt.

🔁 Attempt 3 — Running Simulated Annealing...
3 1 4 5 5 8 7 5 6
1 1 1 1 2 7 8 3 1
8 7 5 3 1 4 4 2 9
1 1 7 1 8 6 5 6 3
2 4 8 1 1 1 1 9 7
9 1 6 1 4 4 2 1 4
6 1 3 9 3 5 4 7 1
5 8 9 6 3 1 1 4 5
4 5 1 8 9 9 3 2 2
Final Conflicts: 61
❌ Could not solve the puzzle on this attempt.

🔁 Attempt 4 — Running Simulated Annealing...
3 9 4 5 2 8 7 5 6
5 6 5 4 6 7 8 3 5
8 7 5 3 1 6 4 4 9
8 1 7 4 2 1 5 6 3
2 4 3 1 8 1 1 9 7
9 1 6 4 8 4 2 1 4
6 3 1 4 2 5 4 7 8
1 8 9 